In [1]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [2]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: https://compute.procedural.build - Authenticated: Yes


In [3]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 250f85e4
{
    "uid": "250f85e4-9e48-413c-a91f-34e9f702cdab",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "dd160a6f-68a8-468a-817e-4482293fe946",
    "path": "/mnt/cluster/projects/250f85e4-9e48-413c-a91f-34e9f702cdab",
    "path_type": "uuid",
    "company": "f2fe6d6b-bb21-4812-9dc5-12991c46f437"
}


In [4]:
task_name = "Jupyter VWT"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: 1e9dcaf7
{
    "uid": "1e9dcaf7-32de-4621-a180-c29b06099a77",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T12:47:28.145885",
    "config": {
        "case_dir": "foam",
        "task_type": "parent"
    },
    "cluster_base_dir": "/mnt/cluster/projects/250f85e4-9e48-413c-a91f-34e9f702cdab/1e9dcaf7-32de-4621-a180-c29b06099a77",
    "parent": null,
    "meta_json": {},
    "project": "250f85e4-9e48-413c-a91f-34e9f702cdab",
    "name": "Jupyter VWT",
    "dependent_on": null
}


In [5]:
stl_path = os.path.join(data_path, "mesh_vwt", "cfdGeom.stl")
file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/cfdGeom.stl/",
                            files=open(stl_path, 'r').read())
print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl


In [6]:
write_mesh_path = os.path.join(data_path, "mesh_vwt", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "mesh_vwt", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: f9c39423
{
    "uid": "f9c39423-a4e9-49c4-a0c3-dac22af8cc78",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T12:47:29.071997",
    "config": {
        "cmd": "cfd.io.tasks.write_mesh",
        "case_dir": "foam",
        "base_mesh": {
            "type": "simpleBox",
            "cell_size": 2,
            "parameters": {
                "z0": "True",
                "square": "True"
            },
            "bounding_box": {
                "max": [
                    35,
                    35,
                    24
                ],
                "min": [
                    -35,
                    -35,
                    0
                ]
            }
        },
        "task_type": "magpy",
        "snappyhex_mesh": {
            "surfaces": {
                "_269c2d39-4173-45a4-8f97-a21f3917d174": {
                    "level": {
                        "max": "2",
                        "min": "2"
       

In [7]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": cpus,
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ]},
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: 0754a484
{
    "uid": "0754a484-bce9-409d-a12a-2b3f364f433e",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T12:47:29.910203",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ],
        "task_type": "cfd"
    },
    "cluster_base_dir": "/mnt/cluster/projects/250f85e4-9e48-413c-a91f-34e9f702cdab/1e9dcaf7-32de-4621-a180-c29b06099a77",
    "parent": {
        "uid": "1e9dcaf7-32de-4621-a180-c29b06099a77",
        "name": "Jupyter VWT"
    },
    "meta_json": {},
    "project": "250f85e4-9e48-413c-a91f-34e9f702cdab",
    "name": "Mesh",
    "dependent_on": {
        "uid": "f9c39423-a4e9-49c4-a0c3-dac22af8cc78",
        "name": "Actions"
    }
}


In [8]:
payload_vwt = {
    "name": "VWT",
    "config": {
        "task_type": "cfd",
        "cmd": "wind_tunnel",
        "commands": [
            0,
            90,
            180,
            270
        ],
        "cpus": cpus,
        "iterations": {
            "init": 100,
            "run": 50
        }
    },
    "parent": parent_id,
    "dependent_on": mesh_id
}

vwt_task = client.create(f"/api/project/{project_id}/task/", payload_vwt)
vwt_id = vwt_task.get("uid")

print(f"VWT TASK: {vwt_id[:8]}")
print(json.dumps(vwt_task, indent=4))

VWT TASK: 55299fd2
{
    "uid": "55299fd2-06ef-43cd-aa8f-d9459eadec7b",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T12:47:30.222152",
    "config": {
        "cmd": "wind_tunnel",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            0,
            90,
            180,
            270
        ],
        "task_type": "parent",
        "iterations": {
            "run": 50,
            "init": 100
        }
    },
    "cluster_base_dir": "/mnt/cluster/projects/250f85e4-9e48-413c-a91f-34e9f702cdab/1e9dcaf7-32de-4621-a180-c29b06099a77",
    "parent": {
        "uid": "1e9dcaf7-32de-4621-a180-c29b06099a77",
        "name": "Jupyter VWT"
    },
    "meta_json": {},
    "project": "250f85e4-9e48-413c-a91f-34e9f702cdab",
    "name": "VWT",
    "dependent_on": null
}


In [9]:
payload = {
    "name": "Probing",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [1, 1, 1],
        "sets": [
            {"name": "set1",
             "points": [
                 [0, 0, 1],
                 [0, 1, 1]
             ]}
        ],
        "fields": ["U", "p"]
    },
    "parent": parent_id,
    "dependent_on": vwt_id
}
probe_task = client.create(f"/api/project/{project_id}/task/", payload)
probe_id = probe_task.get("uid")

print(f"PROBE TASK: {probe_id[:8]}")
print(json.dumps(probe_task, indent=4))


PROBE TASK: f3f9aba1
{
    "uid": "f3f9aba1-67b8-411e-8027-834d324ad181",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T12:47:30.969241",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            1,
            1,
            1
        ],
        "sets": [
            {
                "name": "set1",
                "points": [
                    [
                        0,
                        0,
                        1
                    ],
                    [
                        0,
                        1,
                        1
                    ]
                ]
            }
        ],
        "fields": [
            "U",
            "p"
        ],
        "case_dir": "VWT/",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "task_type": "cfd"
    },
    "cluster_base_dir": "/mnt/cluster/projects/250f85e4-9e48-

In [10]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")